In [9]:
# Importing required packages

import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from getpass import getpass
import mysql.connector as connection
from functools import reduce
import datetime
from datetime import datetime


pd.set_option('display.max_column', None)

# The main report generation function

def generate_village_level_report():
    
    # Define a function to connect to MySQL DB
    def f_connect_to_database(p_host, p_user, p_password, p_database):
        while True:
            try:
                # Setup database connection
                v_mydb = connection.connect(
                    host=p_host,
                    user=p_user,
                    password=p_password,
                    database=p_database,
                    use_pure=True
                )
                v_cursor = v_mydb.cursor()
                print("Connected to the database\n")
                return v_mydb, v_cursor

            except connection.Error as err:
                print(f"Error: {err}")
                print("\n retry !")
                return None, None
           
    # Define a function to close the database connection
    def f_close_database_connection(p_mydb, p_cursor):
        try:
            if p_cursor is not None:
                p_cursor.close()
                print("Cursor closed.")
            if p_mydb.is_connected():
                p_mydb.close()
                print("Database connection closed.\n")
        except Exception as e:
            return f"An error occurred while closing the database connection: {e}"
    
    # Function to execute a query
    def f_execute_query_with_params(p_mydb, p_cursor, p_query):
        try:
            if p_mydb.is_connected():
                p_cursor.execute(p_query)
                v_results = p_cursor.fetchall()
                v_columns = [desc[0] for desc in p_cursor.description]
                df = pd.DataFrame(v_results, columns=v_columns)
                return df
            else:
                print("Database connection is not opened.")
                return None
        except Exception as e:
            print(f"An error occurred: {e}")
            return None

    
    # Function to execute each query and store the output in a dictionary
    def f_execute_queries_and_collect_results(v_mydb, v_cursor, v_queries):
        v_output_dataframes = {}

        for query_name, query in v_queries.items():
            try:
                df = f_execute_query_with_params(v_mydb, v_cursor, query)

                if df is not None:
                    v_output_dataframes[query_name] = df
                    print(f"{query_name} executed successfully.\n")
            except Exception as e:
                print(f"An error occurred while executing {query_name}: {e}")
                return False

        return v_output_dataframes

    
    # Function to merge dataframes
    def f_merge_dataframes(v_output_dataframes):
        try:
            merged_df = None

            for df in v_output_dataframes.values():
                if merged_df is None:
                    merged_df = df
                else:
                    merged_df = pd.merge(merged_df, df, on='location_id', how='left')

            return merged_df.copy()
        except Exception as e:
            print(f"An error occurred while merging dataframes: {e}")
            return None

    # Function to check database connection
    def f_check_database_connection(v_mydb):
        try:
            if v_mydb.is_connected():
                print("Connected")
                return True
            else:
                print("Not connected")
                return False
        except Exception as e:
            print(f"An error occurred while checking database connection: {e}")
            return False

    # Function to export the resulting DataFrame to Excel
    def f_export_to_excel(df, excel_file_path):
        try:
            df.to_excel(excel_file_path, index=False)
            print(f"DataFrame exported to {excel_file_path} successfully!")
        except Exception as e:
            print(f"An error occurred while exporting to Excel: {e}")    
    
    # Input parameters
    v_host_input = input("Enter the host: ")
    v_user_input = input("Enter the username: ")
    v_password_input = getpass("Enter the password: ")  # Use getpass() function directly
    v_database_input = input("Enter the database: ")

    # SQL queries
    v_queries = {'Query1': '''select p.patient_id,
                                    p.creator,
                                    u.username as Sevika_Name,
                                    pi.location_id,
                                    pa.value as hh_id
                            from    patient p,
                                    users u,
                                    patient_identifier pi,
                                    person_attribute pa 
                            where   p.voided = 0
                            and     u.user_id = p.creator
                            and     pi.patient_id = p.patient_id 
                            and     pi.voided = 0 
                            and     pa.voided = 0 
                            and     pa.person_attribute_type_id = 24 
                            and     pa.person_id = p.patient_id;''',

                 'Query2': '''select  l.location_id,
                                    l.name as Village,
                                    ls.name as Sanch,
                                    ld.name as District,
                                    l.state_province as State,
                                    la.value_reference as Village_Type
                            from    location l
                            join    location_tag_map lmap on (lmap.location_id = l.location_id and lmap. location_tag_id = 8)
                            left join location ls on (ls.location_id = l.parent_location)
                            left join location ld on (ld.location_id = ls.parent_location)
                            left join location_attribute la on (la.voided = 0 and la.attribute_type_id = 11 and la.location_id = l.location_id);'''}

    # Connect to the database
    v_mydb, v_cursor = f_connect_to_database(v_host_input, v_user_input, v_password_input, v_database_input)

    # Executing SQL queries
    v_output_dataframes = f_execute_queries_and_collect_results(v_mydb, v_cursor, v_queries)

    # Close database connection
    f_close_database_connection(v_mydb, v_cursor)

    # Calling merge function
    v_sample = f_merge_dataframes(v_output_dataframes)

    # Check database connection
    is_connected = f_check_database_connection(v_mydb)

    # Remove duplicates
    v_sample.drop_duplicates(inplace=True)

    # Fill missing values
    v_sample.fillna('', inplace=True)

    # Aggregating data
    df_new = v_sample.groupby(['Sevika_Name', 'State', 'District', 'Sanch', 'Village', 'Village_Type'], as_index=False).agg({'patient_id': 'count', 'hh_id': 'nunique'})

    # Renaming columns
    df_new = df_new.rename(columns={'patient_id': 'Patient_count', 'hh_id': 'HouseHold_count'})
    
    # Print the head of the final DataFrame
    #print("First 10 rows of the final DataFrame:")
    #if df_new is not None:
    #    print(df_new.head(10))
    #else:
    #    print("No data available in the final DataFrame.")

    # Get the current date and time
    file_name = datetime.now()
    formatted_file_name = int(file_name.strftime("%Y%m%d%H%M%S"))
    
    # Export the resulting DataFrame to Excel
    df_new.to_excel(f'Village_Level_Report_{formatted_file_name}.xlsx', index=False)


In [11]:
generate_village_level_report()

Enter the host: afi.ekalarogya.org
Enter the username: root
Enter the password: ········
Enter the database: openmrs
Connected to the database

Query1 executed successfully.

Query2 executed successfully.

Cursor closed.
Database connection closed.

Not connected
First 10 rows of the final DataFrame:
    Sevika_Name  State  District    Sanch               Village Village_Type  \
0   ASNE0123baa  Assam  Tinsukia  Bordubi    Bahadur Murah line                
1   ASNE0123baa  Assam  Tinsukia  Bordubi  Bordubi Sawtal basti                
2  ASNE0123baha  Assam  Tinsukia  Bordubi    Bahadur 11 no line                
3  ASNE0123baha  Assam  Tinsukia  Bordubi           Jengni Gaon                
4  ASNE0123baha  Assam  Tinsukia  Bordubi   Mankhushi 4 no line                
5   ASNE0123bha  Assam  Tinsukia  Bordubi      Bahadur Kasomari                
6   ASNE0123bor  Assam  Tinsukia  Bordubi  Bordubi Sawtal basti                
7   ASNE0123deh  Assam  Tinsukia  Bordubi    Deohal Raitok